# Walmart Sales Insights Dashbaord - Working File

## Import Python Libraries

In [67]:
import gspread_pandas as gsp

import pandas as pd
import numpy as np
import re
from tabulate import tabulate
import datetime

import matplotlib.pyplot as plt 
%matplotlib inline

import seaborn as sns

## Import Data From Google Sheets and Excel Sheet

In [68]:
branch_details = pd.read_excel('Branch_Details.xlsx') # Excel Sheet

order_file = gsp.Spread('1fBU8Mi6_UnPE0jer9gQYDw8WiGSRYNKy0GuEObRu0I8')
order_details = order_file.sheet_to_df(sheet = 'Order_Details', index = 0, header_rows = 1) # google sheet

product_file = gsp.Spread('1evz8m-kvzQdxXyQNueFTGl7oy-ZEp2BxGF9JJ3cz3E4')
product_details = product_file.sheet_to_df(sheet = 'Product_Details', index = 0, header_rows = 1)

User_file = gsp.Spread('1AVGlmdBufGzDZ_9jI4WkzAFkdOFtz20_L8-LsVNaGzk')
user_details = User_file.sheet_to_df(sheet = 'User_Details', index = 0, header_rows = 1)



## Data Cleaning

In [69]:
# Data Frame Columns cleanup for special characters

print(f'Branch details columns before cleanup : {branch_details.columns}')
branch_details.columns = list(map(lambda x:re.sub(r'\W+','_',x).lower().strip('_'), branch_details.columns))
print(f'Branch details columns after cleanup : {branch_details.columns}')
print('\033[1m' + "-------------------------------------------------------------------------------------------------------------------------" + '\033[0m')


print(f'Order details columns before cleanup : {order_details.columns}')
order_details.columns = list(map(lambda x:re.sub(r'\W+','_',x).lower().strip('_'), order_details.columns))
print(f'Order details columns after cleanup : {order_details.columns}')
print('\033[1m' + "-------------------------------------------------------------------------------------------------------------------------"  + '\033[0m')


print(f'Product details columns before cleanup : {product_details.columns}')
product_details.columns = list(map(lambda x:re.sub(r'\W+','_',x).lower().strip('_'), product_details.columns))
print(f'Product details columns after cleanup : {product_details.columns}')
print('\033[1m' + "-------------------------------------------------------------------------------------------------------------------------"  + '\033[0m')


print(f'User details columns before cleanup : {user_details.columns}')
user_details.columns = list(map(lambda x:re.sub(r'\W+','_',x).lower().strip('_'), user_details.columns))
print(f'User details columns after cleanup : {user_details.columns}')
print('\033[1m' + "-------------------------------------------------------------------------------------------------------------------------"  + '\033[0m')

Branch details columns before cleanup : Index(['Branch_ID', 'Branch_Name', 'Location@', 'Branch_Manager'], dtype='object')
Branch details columns after cleanup : Index(['branch_id', 'branch_name', 'location', 'branch_manager'], dtype='object')
-------------------------------------------------------------------------------------------------------------------------
Order details columns before cleanup : Index(['Order_ID', 'Date', 'Quantity', 'Branch_ID', 'Product_ID', 'user_id',
       'Sale'],
      dtype='object')
Order details columns after cleanup : Index(['order_id', 'date', 'quantity', 'branch_id', 'product_id', 'user_id',
       'sale'],
      dtype='object')
-------------------------------------------------------------------------------------------------------------------------
Product details columns before cleanup : Index(['Product_ID', 'Product_Category', 'Product_name', 'Price', 'Stock'], dtype='object')
Product details columns after cleanup : Index(['product_id', 'product_ca

Find Empty rows/columns and delete them

In [70]:
# Rows and Columns in dataframe before deleteing empty columns

print('\nDataset before empty columns cleanup : \n')
print(f'Branch dataset has {branch_details.shape[0]} rows and {branch_details.shape[1]} columns')
print(f'Order dataset has {order_details.shape[0]} rows and {order_details.shape[1]} columns')
print(f'Product dataset has {product_details.shape[0]} rows and {product_details.shape[1]} columns')
print(f'User dataset has {user_details.shape[0]} rows and {user_details.shape[1]} columns')


Dataset before empty columns cleanup : 

Branch dataset has 17 rows and 4 columns
Order dataset has 1037 rows and 7 columns
Product dataset has 7118 rows and 5 columns
User dataset has 11176 rows and 9 columns


In [71]:
branch_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   branch_id       17 non-null     object 
 1   branch_name     17 non-null     object 
 2   location        16 non-null     object 
 3   branch_manager  0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 676.0+ bytes


In [72]:
order_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1037 entries, 0 to 1036
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   order_id    1037 non-null   object
 1   date        1037 non-null   object
 2   quantity    1037 non-null   object
 3   branch_id   1037 non-null   object
 4   product_id  1037 non-null   object
 5   user_id     1037 non-null   object
 6   sale        1037 non-null   object
dtypes: object(7)
memory usage: 56.8+ KB


In [73]:
product_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7118 entries, 0 to 7117
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_id        7118 non-null   object
 1   product_category  7118 non-null   object
 2   product_name      7118 non-null   object
 3   price             7118 non-null   object
 4   stock             7118 non-null   object
dtypes: object(5)
memory usage: 278.2+ KB


In [74]:
user_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11176 entries, 0 to 11175
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   user_id         11176 non-null  object
 1   cust_name       11176 non-null  object
 2   gender          11176 non-null  object
 3   age_group       11176 non-null  object
 4   age             11176 non-null  object
 5   marital_status  11176 non-null  object
 6   state           11176 non-null  object
 7   zone            11176 non-null  object
 8   occupation      11176 non-null  object
dtypes: object(9)
memory usage: 785.9+ KB


In [75]:
# Delete columns which are completely blank

branch_details.dropna(axis=1, how='all', inplace=True)
order_details.dropna(axis=1, how='all', inplace=True)
product_details.dropna(axis=1, how='all', inplace=True)
user_details.dropna(axis=1, how='all', inplace=True)

In [76]:
# Delete rows which has certain field as blank

branch_details = branch_details[(branch_details.location.notnull()) & (branch_details.location)]

order_details = order_details[order_details.quantity.notnull() & (order_details.quantity) & (order_details.quantity != 0)]

product_details = product_details[product_details.price.notnull() & (product_details.price) & (product_details.price != 0)]


In [77]:
# Rows and Columns in dataframe after deleteing empty columns

print('\nDataset after empty columns cleanup : \n')
print(f'Branch dataset has {branch_details.shape[0]} rows and {branch_details.shape[1]} columns')
print(f'Order dataset has {order_details.shape[0]} rows and {order_details.shape[1]} columns')
print(f'Product dataset has {product_details.shape[0]} rows and {product_details.shape[1]} columns')
print(f'User dataset has {user_details.shape[0]} rows and {user_details.shape[1]} columns')


Dataset after empty columns cleanup : 

Branch dataset has 16 rows and 3 columns
Order dataset has 1034 rows and 7 columns
Product dataset has 7118 rows and 5 columns
User dataset has 11176 rows and 9 columns


In [78]:
# pd.isnull(product_details).sum()

In [79]:
# Keep only numeric values

In [80]:
order_details = order_details[order_details['quantity'].str.isnumeric()]

product_details = product_details[product_details['price'].str.isnumeric()]
product_details = product_details[product_details['stock'].str.isnumeric()]

user_details = user_details[user_details['age'].str.isnumeric()]

## Change data type if required

In [81]:
order_details['quantity'] = order_details['quantity'].astype('int')

product_details['price'] = product_details['price'].astype('int')
product_details['stock'] = product_details['stock'].astype('int')

user_details['age'] = user_details['age'].astype('int')

In [82]:
order_details.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1034 entries, 0 to 1036
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   order_id    1034 non-null   object
 1   date        1034 non-null   object
 2   quantity    1034 non-null   int64 
 3   branch_id   1034 non-null   object
 4   product_id  1034 non-null   object
 5   user_id     1034 non-null   object
 6   sale        1034 non-null   object
dtypes: int64(1), object(6)
memory usage: 64.6+ KB


In [83]:
product_details.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7117 entries, 0 to 7117
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_id        7117 non-null   object
 1   product_category  7117 non-null   object
 2   product_name      7117 non-null   object
 3   price             7117 non-null   int64 
 4   stock             7117 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 333.6+ KB


In [84]:
user_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11176 entries, 0 to 11175
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   user_id         11176 non-null  object
 1   cust_name       11176 non-null  object
 2   gender          11176 non-null  object
 3   age_group       11176 non-null  object
 4   age             11176 non-null  int64 
 5   marital_status  11176 non-null  object
 6   state           11176 non-null  object
 7   zone            11176 non-null  object
 8   occupation      11176 non-null  object
dtypes: int64(1), object(8)
memory usage: 785.9+ KB


In [85]:
product_details.rename(columns={'stock':'available_stock'}, inplace = True)